In [4]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Embedding
from tensorflow import keras
import tensorflow_datasets as tfds


In [5]:
(train_data, test_data), info = tfds.load(
    'imdb_reviews',
    split=[tfds.Split.TRAIN, tfds.Split.TEST],
    as_supervised=True,
    with_info=True
)
tokenizer = keras.preprocessing.text.Tokenizer(num_words=10000, oov_token='<OOV>')
train_sentences, train_labels = zip(*[(sent.numpy().decode('utf8'), label.numpy()) for sent, label in train_data])
tokenizer.fit_on_texts(train_sentences)
train_sequences = tokenizer.texts_to_sequences(train_sentences)
train_padded = keras.preprocessing.sequence.pad_sequences(train_sequences, maxlen=100, padding='post', truncating='post')

test_sentences, test_labels = zip(*[(sent.numpy().decode('utf8'), label.numpy()) for sent, label in test_data])
test_sequences = tokenizer.texts_to_sequences(test_sentences)
test_padded = keras.preprocessing.sequence.pad_sequences(test_sequences, maxlen=100, padding='post', truncating='post')

train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

In [6]:
model = keras.models.Sequential([
    keras.layers.Embedding(10000, 64),
    keras.layers.SimpleRNN(64),
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

history = model.fit(train_padded, train_labels, epochs=10, validation_data=(test_padded, test_labels))

Epoch 1/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 33s 40ms/step - accuracy: 0.5183 - loss: 0.6932 - val_accuracy: 0.6602 - val_loss: 0.6436
Epoch 2/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 32s 41ms/step - accuracy: 0.6544 - loss: 0.6328 - val_accuracy: 0.7080 - val_loss: 0.5912
Epoch 3/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 35s 45ms/step - accuracy: 0.6881 - loss: 0.5939 - val_accuracy: 0.5420 - val_loss: 0.7260
Epoch 4/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 36s 45ms/step - accuracy: 0.6798 - loss: 0.5864 - val_accuracy: 0.5547 - val_loss: 0.7235
Epoch 5/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 41s 45ms/step - accuracy: 0.7453 - loss: 0.5125 - val_accuracy: 0.6820 - val_loss: 0.6458
Epoch 6/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 36s 46ms/step - accuracy: 0.7807 - loss: 0.4561 - val_accuracy: 0.5363 - val_loss: 0.7495
Epoch 7/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 37s 41ms/step - accuracy: 0.7481 - loss: 0.4926 - val_accuracy: 0.5328 - val_loss: 0.7444
Epoch 8/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 31s 40ms/step - accuracy: 0.7192 - loss: 0.5249 - 

In [8]:
score = model.evaluate(test_padded, test_labels, verbose=0)
print(f"Test accuracy: {score[1]:.2f}")

Test accuracy: 0.66


In [10]:
def predict_sentiment(review_text):
    text = review_text.lower()
    text = re.sub(r'[^a-z0-9\s]', '', text)

    seq = tokenizer.texts_to_sequences([text])
    padded = pad_sequences(seq, maxlen=200)

    prediction = model.predict(padded)[0][0]
    return f"{'Positive' if prediction >= 0.5 else 'Negative'} (Probability: {prediction:.2f})"

sample_review = "great movie."
print(f"Review: {sample_review}")
print(f"Sentiment: {predict_sentiment(sample_review)}")

Review: great movie.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
Sentiment: Negative (Probability: 0.15)
